# Code for importing iPython notebooks

In [1]:
%reset -f
import io, os, sys, types
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = current.read(f, 'json')


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.worksheets[0].cells:
                if cell.cell_type == 'code' and cell.language == 'python':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.input)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())


C:\Users\David Castineira\Anaconda2\lib\site-packages\nbformat\current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


# Import and Run Pipeline

In [2]:
import scipy
import numpy
import pandas
import csv
import MergeDiagnosis_AdjustedLabels
import Categorical_Updated
import LongitudinalDataAnalysis
import Imputation
import FeatureReduction
import SupervisedLearning
import ModelPerformance
import TrainTestSplit

importing Jupyter notebook from MergeDiagnosis_AdjustedLabels.ipynb
importing Jupyter notebook from Categorical_Updated.ipynb
importing Jupyter notebook from LongitudinalDataAnalysis.ipynb


C:\Users\David Castineira\Anaconda2\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


<IPython.core.display.Javascript object>

importing Jupyter notebook from Imputation.ipynb
importing Jupyter notebook from FeatureReduction.ipynb


<IPython.core.display.Javascript object>

importing Jupyter notebook from SupervisedLearning.ipynb
importing Jupyter notebook from ModelPerformance.ipynb
importing Jupyter notebook from TrainTestSplit.ipynb


In [3]:
# -------------------------------
# Merge Data
# -------------------------------
merged_data = MergeDiagnosis_AdjustedLabels.data_preprocess(study = "all",imaging_to_drop = 'all', reversions = 'label0')

In [4]:
# -------------------------------
# Categorical to Numerical
# -------------------------------
date_cols = ['update_stamp','EXAMDATE','EXAMDATE_bl']
cols_to_ignore = ['PTID']

Categorical_Updated.categorical_conversion(date_cols,cols_to_ignore)


Columns that are one-hot encoded
-------------------------------------
['VISCODE', 'COLPROT', 'ORIGPROT', 'DX_bl', 'PTGENDER', 'PTETHCAT', 'PTRACCAT', 'PTMARRY']


In [5]:
# -------------------------------
# Longitudinal Data Analysis
# -------------------------------

# Input file name for Longitudinal Data Analysis
InputToLongitudinal='CategoricalToNumerical.csv'

with open(InputToLongitudinal) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    labels = next(reader)

# Output file name from this script
OutputFromLongitudinal='LongitudinalDataAnalysis.csv'

# Patient RID Features
Patient_FEATURES=['RID'];

# Demographic Features
Demo_FEATURES_type=['AGE','PTEDUCAT','PTGENDER','PTETHCAT','PTRACCAT','PTMARRY']
Demo_FEATURES = []
for i in labels:
    if i in Demo_FEATURES_type:
        Demo_FEATURES.append(i)
    elif i.find("_") != -1 and i[:i.find("_")] in Demo_FEATURES_type:
        Demo_FEATURES.append(i)

# Baseline OneTime Features
BaselineOneTime_FEATURES_type = ['APOE4','Years_bl','ORIGPROT']
BaselineOneTime_FEATURES = []
for i in labels:
    if i in BaselineOneTime_FEATURES_type:
        BaselineOneTime_FEATURES.append(i)
    elif i.rfind("_") != -1 and i[:i.rfind("_")] in BaselineOneTime_FEATURES_type:
        BaselineOneTime_FEATURES.append(i)

# Time Headers
Time_FEATURES_type=['SITE','Month','update_stamp_minus_EXAMDATE_bl','update_stamp_minus_EXAMDATE','EXAMDATE_minus_EXAMDATE_bl',
               'COLPROT','M','Month_bl']
Time_FEATURES = []
for i in labels:
    if i in Time_FEATURES_type:
        Time_FEATURES.append(i)
    elif i.find("_") != -1 and i[:i.find("_")] in Time_FEATURES_type:
        Time_FEATURES.append(i)
Time_FEATURES.insert(len(Time_FEATURES), Time_FEATURES.pop(Time_FEATURES.index('Month_bl'))) # Month_bl must be the last time feature

# Baseline Evaluation Features
BaselineEvaluation_FEATURES=['CDRSB_bl','ADAS11_bl','ADAS13_bl','MMSE_bl','RAVLT_learning_bl','RAVLT_forgetting_bl',
                             'RAVLT_perc_forgetting_bl','RAVLT_immediate_bl','FAQ_bl','MOCA_bl','EcogPtLang_bl','EcogPtVisspat_bl',
                             'EcogPtPlan_bl','EcogPtOrgan_bl','EcogPtDivatt_bl','EcogPtMem_bl','EcogPtTotal_bl','EcogSPLang_bl',
                             'EcogSPVisspat_bl','EcogSPPlan_bl','EcogSPOrgan_bl','EcogSPDivatt_bl','EcogSPMem_bl','EcogSPTotal_bl'];



# Current Medical Evaluation
CurrentEvaluation_FEATURES=['CDRSB','ADAS11','ADAS13','MMSE','RAVLT_learning','RAVLT_forgetting','RAVLT_perc_forgetting','RAVLT_immediate',
                            'FAQ','MOCA','EcogPtLang','EcogPtVisspat','EcogPtPlan','EcogPtOrgan','EcogPtDivatt','EcogPtMem','EcogPtTotal',
                            'EcogSPLang','EcogSPVisspat','EcogSPPlan','EcogSPOrgan','EcogSPDivatt','EcogSPMem','EcogSPTotal'];


# Current Diagnosis
CurrentDiagnosis_FEATURES= ['AD'];



In [6]:
res = []

for i in range(100):
    
    print "###########################################"; 
    print "###########################################";
    print i;
    print "###########################################";
    print "###########################################";
    
    
    # -------------------------------
    # Longitudinal Data Analysis
    # -------------------------------

    # Longitudinal Method
    LongitudinalMethod=2;
    MetricList=['MaxTime','Delta','Mean','Std'];

    # Run Longitudinal Data Anaysis
    LongitudinalDataAnalysis.runLongitudinal(InputToLongitudinal,OutputFromLongitudinal,Patient_FEATURES,Demo_FEATURES,\
                                             BaselineOneTime_FEATURES,Time_FEATURES,BaselineEvaluation_FEATURES,\
                                             CurrentEvaluation_FEATURES,CurrentDiagnosis_FEATURES,LongitudinalMethod,MetricList)

    # # !!!!!!!!!!!!!!!!!!!!!!--------------------------------------------------------------
    # Keep only a few columns 
    # --------------------------------------------------------------
    import pandas as pd
    f=pd.read_csv("LongitudinalDataAnalysis.csv")
    keep_col = ['CDRSB_MaxTime','CDRSB_Delta','CDRSB_Mean','CDRSB_Std',\
                'ADAS13_MaxTime','ADAS13_Delta','ADAS13_Mean','ADAS13_Std',\
                'MOCA_MaxTime','MOCA_Delta','MOCA_Mean','MOCA_Std',\
                'Diagnostics']
    new_f = f[keep_col]
    new_f.to_csv("LongitudinalDataAnalysis.csv", index=False)
    
    
    # ------------------
    # Train Test Split
    # ------------------
    TrainTestSplit.traintest_split(0.33)


    
    # ----------------------------------------------------
    # Limit number of months available for test patients
    # ----------------------------------------------------
    # Longitudinal Method
    LongitudinalMethod=3;

    # Run Longitudinal Data Anaysis
    LongitudinalDataAnalysis.runLongitudinal(InputToLongitudinal,OutputFromLongitudinal,Patient_FEATURES,Demo_FEATURES,\
                                             BaselineOneTime_FEATURES,Time_FEATURES,BaselineEvaluation_FEATURES,\
                                             CurrentEvaluation_FEATURES,CurrentDiagnosis_FEATURES,LongitudinalMethod,MetricList)

         
    # !!!!!!!!!!!!!!!!!!!!!! --------------------------------------------------------------
    # Keep only a few columns 
    # --------------------------------------------------------------
    f=pd.read_csv("LongitudinalDataAnalysis.csv")
    new_f = f[keep_col]
    new_f.to_csv("LongitudinalDataAnalysis.csv", index=False)
    
    f2=pd.read_csv("LongitudinalDataAnalysis_test.csv")
    new_f2 = f2[keep_col]
    new_f2.to_csv("LongitudinalDataAnalysis_test.csv", index=False)
    
    
    # ------------------
    # Imputation
    # ------------------
    #Imputation.imputation('knn')
    Imputation.imputation('meanmode')
    #Imputation.imputation('nuclearnorm')
    #Imputation.imputation('softimpute')

    # ----------------------------------
    # Feature Reduction 
    # ----------------------------------

    # Input and Output files from Feature Reduction for train set
    InputToFeatureReduction_train     ='ImputedMatrix_train.csv'
    OutputFromFeatureReduction_train  ='Features_train.csv'
    InputToFeatureReduction_test      ='ImputedMatrix_test.csv'
    OutputFromFeatureReduction_test   ='Features_test.csv'


    # Normalization method
    NormalizationMethod='MinMax'
    #NormalizationMethod='MeanStd'

    # Feature Reduction Method and Settings
    #FeatureReductionMethod='SVD'; 
    ExplainedVariance=0.99; # For method 'SVD'

    FeatureReductionMethod='none' # 'AffinityPropagation', 'SVD' or 'none' 
    APpreference=-50; # Hyperparameter for method 'AffinityPropagation'


    # Run Feature Reduction
    FeatureReduction.RunFeatureReduction(InputToFeatureReduction_train,OutputFromFeatureReduction_train,\
                                         InputToFeatureReduction_test,OutputFromFeatureReduction_test,\
                                         NormalizationMethod,FeatureReductionMethod,ExplainedVariance,APpreference)


    # ------------------
    # Supervised Learning- Model 3
    # ------------------
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    import numpy as np
    data_train = np.loadtxt('Features_train.csv', delimiter=",", skiprows = 1)
    data_test = np.loadtxt('Features_test.csv', delimiter=",", skiprows = 1)
    training_set_X, test_set_X, training_set_Y, test_set_Y = data_train[:,:-1], data_test[:,:-1], data_train[:,-1], data_test[:,-1]
    best_params = {'n_estimators': 1000}
    model = RandomForestClassifier()
    SupervisedLearning.test_model(training_set_X, test_set_X, training_set_Y, test_set_Y, model, best_params)
    res.append(ModelPerformance.model_performance('all'))

    #------------------
    # Model Evaluation
    # ------------------
    #print ModelPerformance.model_performance('confusion_matrix')
    print ModelPerformance.model_performance('all')

###########################################
###########################################
0
###########################################
###########################################


,Unnamed: 0,RID,SITE,AGE,PTEDUCAT,APOE4,CDRSB,ADAS11,ADAS13,MMSE,...,PTRACCAT_Black,PTRACCAT_Hawaiian/Other PI,PTRACCAT_More than one,PTRACCAT_Unknown,PTRACCAT_White,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Unknown,PTMARRY_Widowed,AD
0,0,2,11,74.3,16,0.0,0.0,10.67,18.67,28.0,...,0,0,0,0,1,1,0,0,0,0
1,1,3,11,81.3,18,1.0,4.5,22.00,31.00,20.0,...,0,0,0,0,1,1,0,0,0,1
2,2,3,11,81.3,18,1.0,6.0,19.00,30.00,24.0,...,0,0,0,0,1,1,0,0,0,1


Input Matrix Size:
-----------------------
(12736L, 104L)
 
Identified columns of interest in input file: 
------------------------------------------------------------------------------------------------------------------------
Patient RID column: [1]
Demo columns: [  3   4  90  91  92  93  94  95  96  97  98  99 100 101 102]
BaselineOneTime columns: [ 5 54 84 85]
BaselineEvaluation columns: [30 31 32 33 35 36 37 34 38 39 41 42 43 44 45 40 46 48 49 50 51 52 47 53]
Time columns: [ 2 56 57 58 59 60 82 83 55]
CurrentEvaluation columns: [ 6  7  8  9 11 12 13 10 14 15 17 18 19 20 21 16 22 24 25 26 27 28 23 29]
CurrentDiagnosis columns: [103]
------
Method 2 for Longitudinal Data Analysis
------
 
New Input Matrix Size:
-----------------------
(1737L, 115L)
 
New Output Matrix Size:
-----------------------
(1737L,)


,Unnamed: 0,RID,SITE,AGE,PTEDUCAT,APOE4,CDRSB,ADAS11,ADAS13,MMSE,...,PTRACCAT_Black,PTRACCAT_Hawaiian/Other PI,PTRACCAT_More than one,PTRACCAT_Unknown,PTRACCAT_White,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Unknown,PTMARRY_Widowed,AD
0,0,2,11,74.3,16,0.0,0.0,10.67,18.67,28.0,...,0,0,0,0,1,1,0,0,0,0
1,1,3,11,81.3,18,1.0,4.5,22.00,31.00,20.0,...,0,0,0,0,1,1,0,0,0,1
2,2,3,11,81.3,18,1.0,6.0,19.00,30.00,24.0,...,0,0,0,0,1,1,0,0,0,1


Input Matrix Size:
-----------------------
(12736L, 104L)
 
Identified columns of interest in input file: 
------------------------------------------------------------------------------------------------------------------------
Patient RID column: [1]
Demo columns: [  3   4  90  91  92  93  94  95  96  97  98  99 100 101 102]
BaselineOneTime columns: [ 5 54 84 85]
BaselineEvaluation columns: [30 31 32 33 35 36 37 34 38 39 41 42 43 44 45 40 46 48 49 50 51 52 47 53]
Time columns: [ 2 56 57 58 59 60 82 83 55]
CurrentEvaluation columns: [ 6  7  8  9 11 12 13 10 14 15 17 18 19 20 21 16 22 24 25 26 27 28 23 29]
CurrentDiagnosis columns: [103]
------
Method 3 for Longitudinal Data Analysis
------
 Total number of test patients: 574
 
New Input Matrix Size:
-----------------------
(1737L, 115L)
 
New Output Matrix Size:
-----------------------
(1737L,)
[0.83544303797468356, 0.6785714285714286, 0.82608695652173914, 0.7450980392156864]
###########################################
################

...

Input Matrix Size:
-----------------------
(12736L, 104L)
 
Identified columns of interest in input file: 
------------------------------------------------------------------------------------------------------------------------
Patient RID column: [1]
Demo columns: [  3   4  90  91  92  93  94  95  96  97  98  99 100 101 102]
BaselineOneTime columns: [ 5 54 84 85]
BaselineEvaluation columns: [30 31 32 33 35 36 37 34 38 39 41 42 43 44 45 40 46 48 49 50 51 52 47 53]
Time columns: [ 2 56 57 58 59 60 82 83 55]
CurrentEvaluation columns: [ 6  7  8  9 11 12 13 10 14 15 17 18 19 20 21 16 22 24 25 26 27 28 23 29]
CurrentDiagnosis columns: [103]
------
Method 3 for Longitudinal Data Analysis
------
 Total number of test patients: 574
 
New Input Matrix Size:
-----------------------
(1737L, 115L)
 
New Output Matrix Size:
-----------------------
(1737L,)
[0.90789473684210531, 0.86956521739130432, 0.83333333333333337, 0.85106382978723405]
###########################################
##############

,Unnamed: 0,RID,SITE,AGE,PTEDUCAT,APOE4,CDRSB,ADAS11,ADAS13,MMSE,...,PTRACCAT_Black,PTRACCAT_Hawaiian/Other PI,PTRACCAT_More than one,PTRACCAT_Unknown,PTRACCAT_White,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Unknown,PTMARRY_Widowed,AD
0,0,2,11,74.3,16,0.0,0.0,10.67,18.67,28.0,...,0,0,0,0,1,1,0,0,0,0
1,1,3,11,81.3,18,1.0,4.5,22.00,31.00,20.0,...,0,0,0,0,1,1,0,0,0,1
2,2,3,11,81.3,18,1.0,6.0,19.00,30.00,24.0,...,0,0,0,0,1,1,0,0,0,1


Input Matrix Size:
-----------------------
(12736L, 104L)
 
Identified columns of interest in input file: 
------------------------------------------------------------------------------------------------------------------------
Patient RID column: [1]
Demo columns: [  3   4  90  91  92  93  94  95  96  97  98  99 100 101 102]
BaselineOneTime columns: [ 5 54 84 85]
BaselineEvaluation columns: [30 31 32 33 35 36 37 34 38 39 41 42 43 44 45 40 46 48 49 50 51 52 47 53]
Time columns: [ 2 56 57 58 59 60 82 83 55]
CurrentEvaluation columns: [ 6  7  8  9 11 12 13 10 14 15 17 18 19 20 21 16 22 24 25 26 27 28 23 29]
CurrentDiagnosis columns: [103]
------
Method 2 for Longitudinal Data Analysis
------
 
New Input Matrix Size:
-----------------------
(1737L, 115L)
 
New Output Matrix Size:
-----------------------
(1737L,)


,Unnamed: 0,RID,SITE,AGE,PTEDUCAT,APOE4,CDRSB,ADAS11,ADAS13,MMSE,...,PTRACCAT_Black,PTRACCAT_Hawaiian/Other PI,PTRACCAT_More than one,PTRACCAT_Unknown,PTRACCAT_White,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Unknown,PTMARRY_Widowed,AD
0,0,2,11,74.3,16,0.0,0.0,10.67,18.67,28.0,...,0,0,0,0,1,1,0,0,0,0
1,1,3,11,81.3,18,1.0,4.5,22.00,31.00,20.0,...,0,0,0,0,1,1,0,0,0,1
2,2,3,11,81.3,18,1.0,6.0,19.00,30.00,24.0,...,0,0,0,0,1,1,0,0,0,1


Input Matrix Size:
-----------------------
(12736L, 104L)
 
Identified columns of interest in input file: 
------------------------------------------------------------------------------------------------------------------------
Patient RID column: [1]
Demo columns: [  3   4  90  91  92  93  94  95  96  97  98  99 100 101 102]
BaselineOneTime columns: [ 5 54 84 85]
BaselineEvaluation columns: [30 31 32 33 35 36 37 34 38 39 41 42 43 44 45 40 46 48 49 50 51 52 47 53]
Time columns: [ 2 56 57 58 59 60 82 83 55]
CurrentEvaluation columns: [ 6  7  8  9 11 12 13 10 14 15 17 18 19 20 21 16 22 24 25 26 27 28 23 29]
CurrentDiagnosis columns: [103]
------
Method 3 for Longitudinal Data Analysis
------
 Total number of test patients: 574
 
New Input Matrix Size:
-----------------------
(1737L, 115L)
 
New Output Matrix Size:
-----------------------
(1737L,)
[0.90909090909090906, 0.875, 0.90322580645161288, 0.88888888888888884]


In [7]:
#print res
res2=np.asarray(res)
print res2

[[ 0.83544304  0.67857143  0.82608696  0.74509804]
 [ 0.9         0.83783784  0.91176471  0.87323944]
 [ 0.90540541  0.8         0.96        0.87272727]
 [ 0.86419753  0.75675676  0.93333333  0.8358209 ]
 [ 0.86363636  0.76923077  0.76923077  0.76923077]
 [ 0.87951807  0.74285714  0.96296296  0.83870968]
 [ 0.83333333  0.72222222  0.86666667  0.78787879]
 [ 0.85714286  0.78787879  0.83870968  0.8125    ]
 [ 0.87179487  0.78947368  0.9375      0.85714286]
 [ 0.82417582  0.71428571  0.80645161  0.75757576]
 [ 0.83116883  0.75        0.87096774  0.80597015]
 [ 0.83529412  0.68571429  0.88888889  0.77419355]
 [ 0.85714286  0.75675676  0.90322581  0.82352941]
 [ 0.87804878  0.81818182  0.87096774  0.84375   ]
 [ 0.77464789  0.73333333  0.73333333  0.73333333]
 [ 0.90217391  0.86111111  0.88571429  0.87323944]
 [ 0.85057471  0.73529412  0.86206897  0.79365079]
 [ 0.91764706  0.87096774  0.9         0.8852459 ]
 [ 0.85263158  0.72972973  0.87096774  0.79411765]
 [ 0.82608696  0.79166667  0.73

In [8]:
np.mean(res2, axis=0)

array([ 0.86716035,  0.78716783,  0.86493082,  0.82178104])

In [9]:

# # ------------------
# # Supervised Learning- options currently are LogisticRegression, RandomForest, MLP, knn
# # ------------------
# from sklearn.neural_network import MLPClassifier
# from sklearn.ensemble import RandomForestClassifier
# import numpy as np
# data_train = np.loadtxt('Features_train.csv', delimiter=",", skiprows = 1)
# data_test = np.loadtxt('Features_test.csv', delimiter=",", skiprows = 1)

# training_set_X, test_set_X, training_set_Y, test_set_Y = data_train[:,:-1], data_test[:,:-1], data_train[:,-1], data_test[:,-1]

# best_params = {'alpha': 0.05, 'hidden_layer_sizes': (100,100,100)}
# model = MLPClassifier()
# SupervisedLearning.test_model(training_set_X, test_set_X, training_set_Y, test_set_Y, model, best_params)

# #------------------
# # Model Evaluation
# # ------------------
# #print ModelPerformance.model_performance('confusion_matrix')
# print ModelPerformance.model_performance('all')